In [ ]:
# read paper_subject_match
import pandas as pd
match = pd.read_csv('paper_subject_match.csv')
# read paper_info
reference = pd.read_csv('paper_info.csv')

In [ ]:
reference.columns = ['id', 'year', 'references', 'subjects']

In [ ]:
###### convert 'references' column to list ######
new_col = []
for i in range(len(reference)):
    words = reference.loc[i,'refereneces']
    words_split = words.split(',')
    for j in range(len(words_split)):
        words_split[j] = words_split[j].strip(" '[]")
    new_col.append(words_split)
    
reference['references'] = new_col
reference.head()

In [ ]:
##### add paper's subject as new col ######
reference['paper_subject'] = match['paper_subject']

In [ ]:
# reset 'id' column as index
reference = reference.set_index('id',)

In [ ]:
###### find subjects of reference ######
reference_subjects = []
row = 1
for i in reference.index:
    references = reference.loc[i,'references']
    index = [x for x in references if x in reference.index.tolist()]
    subjects = reference.loc[index,'paper_subject'].tolist()
    reference_subjects.append(subjects)
    print('row ' + str(row) + ' finished')
    row +=1

reference['reference_subjects'] = reference_subjects
reference.head()          

In [ ]:
###### separate pairs ######
import numpy as np
impact = reference.loc[:,['paper_subject', 'reference_subjects']]
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

impact2 = explode(impact, 'reference_subjects')
impact2.dropna(inplace = True)

In [ ]:
###### occurrence matrix ######
matrix = pd.get_dummies(impact2['reference_subjects']).groupby(impact2['paper_subject']).apply(sum)
matrix

In [ ]:
###### TF-IDF ######
impact3 = impact2.groupby(['paper_subject','reference_subjects']).size().reset_index()
N = len(impact2['paper_subject'].unique())
D = dict(impact3['reference_subjects'].value_counts())

import numpy as np
idf = {}
for d in D:
    idf[d] = np.log( N / D[d] )

In [ ]:
for key in idf:
    if key not in matrix.index:
        continue
    else:
        matrix.loc[key] = matrix.loc[key] * idf[key]
matrix

In [ ]:
###### visualization of matrix ######
%matplotlib inline
import seaborn as sns
sns.heatmap(matrix)